<a href="https://colab.research.google.com/github/Ali-Fartout/Kaggle/blob/main/TPS_Jan_2022/TPS_jan_2022_TreeBased.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Rquierment

In [228]:
! mkdir ~/.kaggle
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json
! chmod 600 ~/.kaggle/kaggle.json
! kaggle competitions download -c tabular-playground-series-jan-2022

mkdir: cannot create directory ‘/root/.kaggle’: File exists
tabular-playground-series-jan-2022.zip: Skipping, found more recently modified local copy (use --force to force download)


In [229]:
import zipfile

zipfile = zipfile.ZipFile('/content/tabular-playground-series-jan-2022.zip','r')
zipfile.extractall()
zipfile.close()

In [230]:
import pandas as pd
import numpy as np

train = pd.read_csv('/content/train.csv',parse_dates=['date'])
test = pd.read_csv('/content/test.csv',parse_dates=['date'])
sub = pd.read_csv('/content/sample_submission.csv')

# Feature engineering

In [231]:
def Encoder(data,feature):
  items = data[feature].value_counts().index.tolist()
  for index_itr,item in enumerate(data[feature]):                                
      enc_val = items.index(item)
      data[feature].iloc[index_itr] = enc_val
  return data


In [232]:
def engineering(data):
  data['year'] = data.date.dt.year
  data['day'] = data.date.dt.day
  data['month'] = data.date.dt.month
  for col in ['country','store','product']:
    Encoder(data,col)
    
  return data

In [233]:
train = engineering(train)
test = engineering(test)

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [234]:
y = train.pop('num_sold')
yTrain = pd.DataFrame(y,columns=['num_sold'])
xTrain = train.drop(columns=['date','row_id'])
xTest = test.drop(columns=['date','row_id'])

# Model

## DecisionTreeRegressor

In [235]:
from sklearn.tree import DecisionTreeRegressor

decisionsTree_reg = DecisionTreeRegressor(max_depth=6)
decisionsTree_reg.fit(xTrain,yTrain)
pred = decisionsTree_reg.predict(xTest)
sub['num_sold'] = pred

In [236]:
sub.to_csv('DecisionTree.csv')

score 13.70273

## DecisionTreeRegressor important features

In [237]:
decisionsTree_reg = DecisionTreeRegressor(max_depth=8)
decisionsTree_reg.fit(xTrain.loc[:,['product','store','month','country']],yTrain)
pred = decisionsTree_reg.predict(xTest.loc[:,['product','store','month','country']])
sub['num_sold'] = pred

In [238]:
sub = pd.DataFrame()
sub['row_id'] = test['row_id']
sub['num_sold'] = pred
sub.to_csv('DecisionTreeFS.csv',index=False )

score 13.26482 

In [239]:
!kaggle competitions submit  -c tabular-playground-series-jan-2022 -f /content/DecisionTreeFS.csv -m "DecisionTreeFS"

100% 153k/153k [00:03<00:00, 40.6kB/s]
Successfully submitted to Tabular Playground Series - Jan 2022

In [240]:
!kaggle competitions submissions  -c tabular-playground-series-jan-2022

fileName            date                 description     status    publicScore  privateScore  
------------------  -------------------  --------------  --------  -----------  ------------  
DecisionTreeFS.csv  2022-03-14 13:01:25  DecisionTreeFS  complete  10.88245     12.59147      
XGB.csv             2022-03-14 12:45:05  XGB             complete  5.90805      6.98849       
XGB.csv             2022-03-14 12:44:06  XGB             complete  5.90805      6.98849       
XGB.csv             2022-03-14 12:41:45  XGB             complete  5.90805      6.98849       
XGB.csv             2022-03-14 12:41:13  XGB             complete  5.90805      6.98849       
XGB.csv             2022-03-14 12:40:36  XGB             complete  5.90805      6.98849       
XGB.csv             2022-03-14 12:37:56  XGB             complete  5.90805      6.98849       
XGB.csv             2022-03-14 12:32:37  XGB             complete  5.90805      6.98849       
XGB.csv             2022-03-14 12:32:13  XGB      

## Random forest

### One hot encoder!

In [241]:
train = pd.read_csv('/content/train.csv',parse_dates=['date'])
test = pd.read_csv('/content/test.csv',parse_dates=['date'])

In [242]:
def OneHotEncoder(data,cols):
    for col in cols:
        col_ohe = pd.get_dummies(data[col], prefix=col)
        data = pd.concat((data, col_ohe), axis=1).drop(col, axis=1)

    data['year'] = data.date.dt.year
    data['day'] = data.date.dt.day
    data['month'] = data.date.dt.month
    data['saturday'] = (data.date.dt.weekday == 5).astype(int)
    data['sunday'] = (data.date.dt.weekday == 6).astype(int)
    data['endofmonth'] = (data.date.dt.day < 19  ).astype(int)
    data['endofyear'] = ((data.date.dt.month == 12) & (data.date.dt.day > 23 )).astype(int) 
    return data


train = OneHotEncoder(train,['product','store','country'])
test = OneHotEncoder(test,['product','store','country'])

In [243]:
y = train.pop('num_sold')
yTrain = pd.DataFrame(y,columns=['num_sold'])
xTrain = train.drop(columns=['date','row_id'])
xTest = test.drop(columns=['date','row_id'])

In [244]:
from sklearn.ensemble import RandomForestRegressor
randomForest_reg = RandomForestRegressor(max_depth=18, random_state=0)
randomForest_reg.fit(xTrain,yTrain)
pred = randomForest_reg.predict(xTest)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


In [245]:
train.head()

,row_id,date,product_Kaggle Hat,product_Kaggle Mug,product_Kaggle Sticker,store_KaggleMart,store_KaggleRama,country_Finland,country_Norway,country_Sweden,year,day,month,saturday,sunday,endofmonth,endofyear
0,0,2015-01-01,0,1,0,1,0,1,0,0,2015,1,1,0,0,1,0
1,1,2015-01-01,1,0,0,1,0,1,0,0,2015,1,1,0,0,1,0
2,2,2015-01-01,0,0,1,1,0,1,0,0,2015,1,1,0,0,1,0
3,3,2015-01-01,0,1,0,0,1,1,0,0,2015,1,1,0,0,1,0
4,4,2015-01-01,1,0,0,0,1,1,0,0,2015,1,1,0,0,1,0


In [246]:
sub = pd.DataFrame()
sub['row_id'] = test['row_id']
sub['num_sold'] = pred
sub.to_csv('RandomForest.csv',index=False )

In [247]:
!kaggle competitions submit  -c tabular-playground-series-jan-2022 -f /content/RandomForest.csv -m "RandomForest"

100% 135k/135k [00:03<00:00, 34.8kB/s]
Successfully submitted to Tabular Playground Series - Jan 2022

In [248]:
!kaggle competitions submissions  -c tabular-playground-series-jan-2022

fileName            date                 description     status    publicScore  privateScore  
------------------  -------------------  --------------  --------  -----------  ------------  
RandomForest.csv    2022-03-14 13:01:36  RandomForest    complete  7.29935      9.67003       
DecisionTreeFS.csv  2022-03-14 13:01:25  DecisionTreeFS  complete  10.88245     12.59147      
XGB.csv             2022-03-14 12:45:05  XGB             complete  5.90805      6.98849       
XGB.csv             2022-03-14 12:44:06  XGB             complete  5.90805      6.98849       
XGB.csv             2022-03-14 12:41:45  XGB             complete  5.90805      6.98849       
XGB.csv             2022-03-14 12:41:13  XGB             complete  5.90805      6.98849       
XGB.csv             2022-03-14 12:40:36  XGB             complete  5.90805      6.98849       
XGB.csv             2022-03-14 12:37:56  XGB             complete  5.90805      6.98849       
XGB.csv             2022-03-14 12:32:37  XGB      

### One hot encoder and important features!

In [249]:
from sklearn.ensemble import RandomForestRegressor
randomForest_reg = RandomForestRegressor(max_depth=12, random_state=0)
randomForest_reg.fit(xTrain.drop(columns=['year','endofmonth']),yTrain)
pred = randomForest_reg.predict(xTest.drop(columns=['year','endofmonth']))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


In [250]:
sub = pd.DataFrame()
sub['row_id'] = test['row_id']
sub['num_sold'] = pred
sub.to_csv('RandomForest.csv',index=False )

In [251]:
!kaggle competitions submit  -c tabular-playground-series-jan-2022 -f /content/RandomForest.csv -m "RandomForest"

100% 156k/156k [00:03<00:00, 40.2kB/s]
Successfully submitted to Tabular Playground Series - Jan 2022

In [252]:
!kaggle competitions submissions  -c tabular-playground-series-jan-2022

fileName            date                 description     status    publicScore  privateScore  
------------------  -------------------  --------------  --------  -----------  ------------  
RandomForest.csv    2022-03-14 13:01:46  RandomForest    complete  5.90648      7.10492       
RandomForest.csv    2022-03-14 13:01:36  RandomForest    complete  7.29935      9.67003       
DecisionTreeFS.csv  2022-03-14 13:01:25  DecisionTreeFS  complete  10.88245     12.59147      
XGB.csv             2022-03-14 12:45:05  XGB             complete  5.90805      6.98849       
XGB.csv             2022-03-14 12:44:06  XGB             complete  5.90805      6.98849       
XGB.csv             2022-03-14 12:41:45  XGB             complete  5.90805      6.98849       
XGB.csv             2022-03-14 12:41:13  XGB             complete  5.90805      6.98849       
XGB.csv             2022-03-14 12:40:36  XGB             complete  5.90805      6.98849       
XGB.csv             2022-03-14 12:37:56  XGB      

## Catboost

In [253]:
train = pd.read_csv('/content/train.csv',parse_dates=['date'])
test = pd.read_csv('/content/test.csv',parse_dates=['date'])

In [254]:
def OneHotEncoder(data,cols):
    for col in cols:
        col_ohe = pd.get_dummies(data[col], prefix=col)
        data = pd.concat((data, col_ohe), axis=1).drop(col, axis=1)

    data['year'] = data.date.dt.year
    data['day'] = data.date.dt.day
    data['month'] = data.date.dt.month
    data['saturday'] = (data.date.dt.weekday == 5).astype(int)
    data['sunday'] = (data.date.dt.weekday == 6).astype(int)
    data['endofmonth'] = (data.date.dt.day < 19  ).astype(int)
    data['endofyear'] = ((data.date.dt.month == 12) & (data.date.dt.day > 23 )).astype(int) 
    return data


train = OneHotEncoder(train,['product','store','country'])
test = OneHotEncoder(test,['product','store','country'])

In [255]:
y = train.pop('num_sold')
yTrain = pd.DataFrame(y,columns=['num_sold'])
xTrain = train.drop(columns=['date','row_id'])
xTest = test.drop(columns=['date','row_id'])

In [256]:
!pip install catboost

In [257]:
from catboost import CatBoostRegressor

catboost_reg = CatBoostRegressor(loss_function= 'RMSE',
                             iterations = 200,
                             learning_rate=0.1, 
                             depth = 7,
                             )
catboost_reg.fit(xTrain.drop(columns=['year','endofmonth']),yTrain)
pred = catboost_reg.predict(xTest.drop(columns=['year','endofmonth']))

0:	learn: 242.7216767	total: 5.5ms	remaining: 1.09s
1:	learn: 221.9586990	total: 10.9ms	remaining: 1.08s
2:	learn: 203.3890077	total: 15.8ms	remaining: 1.03s
3:	learn: 186.6884249	total: 20.7ms	remaining: 1.01s
4:	learn: 172.1356214	total: 25.6ms	remaining: 1s
5:	learn: 159.1903472	total: 30.4ms	remaining: 984ms
6:	learn: 147.5243166	total: 35.2ms	remaining: 970ms
7:	learn: 137.2559613	total: 39.9ms	remaining: 957ms
8:	learn: 128.2699390	total: 45.5ms	remaining: 965ms
9:	learn: 120.4558308	total: 52.5ms	remaining: 997ms
10:	learn: 113.3755300	total: 57.5ms	remaining: 988ms
11:	learn: 107.2699636	total: 62.2ms	remaining: 975ms
12:	learn: 101.6328209	total: 67.1ms	remaining: 965ms
13:	learn: 96.8866898	total: 71.8ms	remaining: 954ms
14:	learn: 92.6470075	total: 76.7ms	remaining: 946ms
15:	learn: 88.7894765	total: 81.5ms	remaining: 937ms
16:	learn: 85.4107589	total: 86.2ms	remaining: 928ms
17:	learn: 82.5653652	total: 91.1ms	remaining: 921ms
18:	learn: 79.9074499	total: 95.9ms	remaining: 

In [258]:
sub = pd.DataFrame()
sub['row_id'] = test['row_id']
sub['num_sold'] = pred
sub.to_csv('catboost.csv',index=False )

In [259]:
!kaggle competitions submit  -c tabular-playground-series-jan-2022 -f /content/catboost.csv -m "Catboost"

100% 156k/156k [00:02<00:00, 55.5kB/s]
Successfully submitted to Tabular Playground Series - Jan 2022

In [260]:
!kaggle competitions submissions  -c tabular-playground-series-jan-2022

fileName            date                 description     status    publicScore  privateScore  
------------------  -------------------  --------------  --------  -----------  ------------  
catboost.csv        2022-03-14 13:01:56  Catboost        complete  6.02400      7.13826       
RandomForest.csv    2022-03-14 13:01:46  RandomForest    complete  5.90648      7.10492       
RandomForest.csv    2022-03-14 13:01:36  RandomForest    complete  7.29935      9.67003       
DecisionTreeFS.csv  2022-03-14 13:01:25  DecisionTreeFS  complete  10.88245     12.59147      
XGB.csv             2022-03-14 12:45:05  XGB             complete  5.90805      6.98849       
XGB.csv             2022-03-14 12:44:06  XGB             complete  5.90805      6.98849       
XGB.csv             2022-03-14 12:41:45  XGB             complete  5.90805      6.98849       
XGB.csv             2022-03-14 12:41:13  XGB             complete  5.90805      6.98849       
XGB.csv             2022-03-14 12:40:36  XGB      

## LightGBM

In [261]:
from lightgbm import LGBMRegressor
lighgbm_reg = LGBMRegressor(max_depth = 10,learning_rate =0.10,num_leaves =34)
lighgbm_reg.fit(xTrain.drop(columns=['year','endofmonth']),yTrain)
pred = lighgbm_reg.predict(xTest.drop(columns=['year','endofmonth']))

In [262]:
sub = pd.DataFrame()
sub['row_id'] = test['row_id']
sub['num_sold'] = pred
sub.to_csv('LightGBM.csv',index=False )

In [263]:
!kaggle competitions submit  -c tabular-playground-series-jan-2022 -f /content/LightGBM.csv -m "LightGBM"

100% 157k/157k [00:02<00:00, 64.3kB/s]
Successfully submitted to Tabular Playground Series - Jan 2022

In [264]:
!kaggle competitions submissions  -c tabular-playground-series-jan-2022

fileName            date                 description     status    publicScore  privateScore  
------------------  -------------------  --------------  --------  -----------  ------------  
LightGBM.csv        2022-03-14 13:02:02  LightGBM        complete  5.83712      7.14957       
catboost.csv        2022-03-14 13:01:56  Catboost        complete  6.02400      7.13826       
RandomForest.csv    2022-03-14 13:01:46  RandomForest    complete  5.90648      7.10492       
RandomForest.csv    2022-03-14 13:01:36  RandomForest    complete  7.29935      9.67003       
DecisionTreeFS.csv  2022-03-14 13:01:25  DecisionTreeFS  complete  10.88245     12.59147      
XGB.csv             2022-03-14 12:45:05  XGB             complete  5.90805      6.98849       
XGB.csv             2022-03-14 12:44:06  XGB             complete  5.90805      6.98849       
XGB.csv             2022-03-14 12:41:45  XGB             complete  5.90805      6.98849       
XGB.csv             2022-03-14 12:41:13  XGB      

## Xgboost

In [265]:
import xgboost
xgboost_reg = xgboost.XGBRegressor(max_depth=8,n_estimators = 50,learning_rate=0.101,scale_pos_weight = 1,gamma = 0.4,min_child_weight = 0.5)
xgboost_reg.fit(xTrain.drop(columns=['year','endofmonth']),yTrain)
pred = xgboost_reg.predict(xTest.drop(columns=['year','endofmonth']))

[13:02:04] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [266]:
sub = pd.DataFrame()
sub['row_id'] = test['row_id']
sub['num_sold'] = pred
sub.to_csv('XGB.csv',index=False )

In [267]:
!kaggle competitions submit  -c tabular-playground-series-jan-2022 -f /content/XGB.csv -m "XGB"
!kaggle competitions submissions  -c tabular-playground-series-jan-2022

100% 100k/100k [00:03<00:00, 27.9kB/s]
Successfully submitted to Tabular Playground Series - Jan 2022fileName            date                 description     status    publicScore  privateScore  
------------------  -------------------  --------------  --------  -----------  ------------  
XGB.csv             2022-03-14 13:02:12  XGB             complete  5.90805      6.98849       
LightGBM.csv        2022-03-14 13:02:02  LightGBM        complete  5.83712      7.14957       
catboost.csv        2022-03-14 13:01:56  Catboost        complete  6.02400      7.13826       
RandomForest.csv    2022-03-14 13:01:46  RandomForest    complete  5.90648      7.10492       
RandomForest.csv    2022-03-14 13:01:36  RandomForest    complete  7.29935      9.67003       
DecisionTreeFS.csv  2022-03-14 13:01:25  DecisionTreeFS  complete  10.88245     12.59147      
XGB.csv             2022-03-14 12:45:05  XGB             complete  5.90805      6.98849       
XGB.csv             2022-03-14 12:44:06  XGB

## Stacking

In [268]:
from sklearn.ensemble import StackingRegressor

estimators = [
     ('cat', catboost_reg),
    ('xgb', xgboost_reg),
    ('lgbm',lighgbm_reg),
    ('random',randomForest_reg)
     ]

reg = StackingRegressor(estimators=estimators)
reg.fit(xTrain.drop(columns=['year','endofmonth']),yTrain)
pred = reg.predict(xTest.drop(columns=['year','endofmonth']))

/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_stacking.py:758: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0:	learn: 242.7216767	total: 5.76ms	remaining: 1.15s
1:	learn: 221.9586990	total: 11.3ms	remaining: 1.12s
2:	learn: 203.3890077	total: 15.9ms	remaining: 1.04s
3:	learn: 186.6884249	total: 20.9ms	remaining: 1.02s
4:	learn: 172.1356214	total: 25.7ms	remaining: 1s
5:	learn: 159.1903472	total: 30.6ms	remaining: 989ms
6:	learn: 147.5243166	total: 35.4ms	remaining: 975ms
7:	learn: 137.2559613	total: 41.3ms	remaining: 991ms
8:	learn: 128.2699390	total: 46.7ms	remaining: 990ms
9:	learn: 120.4558308	total: 51.3ms	remaining: 976ms
10:	learn: 113.3755300	total: 56.1ms	remaining: 963ms
11:	learn: 107.2699636	total: 60.8ms	remaining: 952ms
12:	learn: 101.6328209	total: 65.7ms	remaining: 945ms
13:	learn: 96.8866898	total: 70.4ms	remaining: 935ms
14:	learn: 92.6470075	total: 75.2ms	remaining: 928ms
15:	learn: 88.7894765	total: 80ms	remaining: 920ms
16:	learn: 85.4107589	total: 84.7ms	remaining: 912ms
17:	learn: 82.5653652	total: 89.5ms	remaining: 905ms
18:	learn: 79.9074499	total: 94.3ms	remaining: 8

In [270]:
sub = pd.DataFrame()
sub['row_id'] = test['row_id']
sub['num_sold'] = pred
sub.to_csv('stacking.csv',index=False )

In [271]:
!kaggle competitions submit  -c tabular-playground-series-jan-2022 -f /content/stacking.csv -m "stacking"
!kaggle competitions submissions  -c tabular-playground-series-jan-2022

100% 156k/156k [00:02<00:00, 57.9kB/s]
Successfully submitted to Tabular Playground Series - Jan 2022fileName            date                 description     status    publicScore  privateScore  
------------------  -------------------  --------------  --------  -----------  ------------  
stacking.csv        2022-03-14 13:04:16  stacking        complete  5.73633      6.95610       
XGB.csv             2022-03-14 13:02:12  XGB             complete  5.90805      6.98849       
LightGBM.csv        2022-03-14 13:02:02  LightGBM        complete  5.83712      7.14957       
catboost.csv        2022-03-14 13:01:56  Catboost        complete  6.02400      7.13826       
RandomForest.csv    2022-03-14 13:01:46  RandomForest    complete  5.90648      7.10492       
RandomForest.csv    2022-03-14 13:01:36  RandomForest    complete  7.29935      9.67003       
DecisionTreeFS.csv  2022-03-14 13:01:25  DecisionTreeFS  complete  10.88245     12.59147      
XGB.csv             2022-03-14 12:45:05  XGB